In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
# To ignore all warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [17]:
df = pd.read_csv(r'framingham.csv')
cols = []
for i in range(len(df.columns)):
    cols.append(df.columns[i].lower().replace(' ', '_'))
df.columns = cols
df

,male,age,education,currentsmoker,cigsperday,bpmeds,prevalentstroke,prevalenthyp,diabetes,totchol,sysbp,diabp,bmi,heartrate,glucose,tenyearchd
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0


In [18]:
df = df.dropna()
df.isna().sum().sum()

0

In [19]:
df.duplicated().sum()

0

In [21]:
df.groupby('tenyearchd').count()

,male,age,education,currentsmoker,cigsperday,bpmeds,prevalentstroke,prevalenthyp,diabetes,totchol,sysbp,diabp,bmi,heartrate,glucose
tenyearchd,,,,,,,,,,,,,,,
0,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099
1,557,557,557,557,557,557,557,557,557,557,557,557,557,557,557


In [22]:
X = df.drop('tenyearchd',axis = 1)
y = df['tenyearchd']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

LR = LogisticRegression()
LR.fit(X_train, y_train)

LR.score(X_test, y_test)

0.8346994535519126

In [26]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

pred = LR.predict(X_test)

print('Precision: ', precision_score(y_test,pred))
print('Recall: ', recall_score(y_test,pred))
print('F1: ', f1_score(y_test,pred))

Precision:  0.5333333333333333
Recall:  0.06557377049180328
F1:  0.11678832116788322


In [27]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.84      0.99      0.91       610
           1       0.53      0.07      0.12       122

    accuracy                           0.83       732
   macro avg       0.69      0.53      0.51       732
weighted avg       0.79      0.83      0.78       732



In [29]:
confusion_matrix(y_test,pred)

array([[603,   7],
       [114,   8]], dtype=int64)

In [31]:
predicted_prob = LR.predict_proba(X_test)[:, 1]
predicted_prob

custom_threshold = 0.3  
custom_predictions = (predicted_prob >= custom_threshold).astype(int)

In [32]:
confusion_matrix(y_test, custom_predictions)

array([[561,  49],
       [ 93,  29]], dtype=int64)

In [33]:
custom_threshold = 0.1  
custom_predictions = (predicted_prob >= custom_threshold).astype(int)

In [34]:
confusion_matrix(y_test,custom_predictions)

array([[287, 323],
       [ 35,  87]], dtype=int64)

In [35]:
print(classification_report(y_test,custom_predictions))

              precision    recall  f1-score   support

           0       0.89      0.47      0.62       610
           1       0.21      0.71      0.33       122

    accuracy                           0.51       732
   macro avg       0.55      0.59      0.47       732
weighted avg       0.78      0.51      0.57       732



In [36]:
df.groupby('tenyearchd').count()

,male,age,education,currentsmoker,cigsperday,bpmeds,prevalentstroke,prevalenthyp,diabetes,totchol,sysbp,diabp,bmi,heartrate,glucose
tenyearchd,,,,,,,,,,,,,,,
0,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099,3099
1,557,557,557,557,557,557,557,557,557,557,557,557,557,557,557


In [37]:
from sklearn.utils import resample

train = pd.concat([X_train, y_train], axis = 1)
train.head()

,male,age,education,currentsmoker,cigsperday,bpmeds,prevalentstroke,prevalenthyp,diabetes,totchol,sysbp,diabp,bmi,heartrate,glucose,tenyearchd
780,0,46,3.0,1,20.0,0.0,0,0,0,253.0,118.0,82.0,19.70,100.0,70.0,0
2218,1,43,2.0,1,20.0,0.0,0,0,0,226.0,132.5,85.0,26.64,72.0,58.0,0
3251,0,48,3.0,1,20.0,0.0,0,0,0,259.0,129.0,81.0,21.08,75.0,65.0,0
3482,1,38,2.0,0,0.0,0.0,0,1,0,256.0,123.0,92.0,25.42,62.0,82.0,0
2662,0,51,1.0,0,0.0,0.0,0,1,0,285.0,151.5,79.0,26.91,84.0,76.0,0


In [38]:
no_CHD = train[train['tenyearchd'] == 0]
yes_CHD = train[train['tenyearchd'] == 1]

display(no_CHD.shape)
display(yes_CHD.shape)

(2489, 16)

(435, 16)

In [39]:
yes_CHD_oversampled = resample(yes_CHD, 
                                    replace = True,
                                    n_samples = len(no_CHD),
                                    random_state = 0)

display(no_CHD.shape)
display(yes_CHD_oversampled.shape)

(2489, 16)

(2489, 16)

In [40]:
train_oversampled = pd.concat([no_CHD, yes_CHD_oversampled])
train_oversampled.head()

,male,age,education,currentsmoker,cigsperday,bpmeds,prevalentstroke,prevalenthyp,diabetes,totchol,sysbp,diabp,bmi,heartrate,glucose,tenyearchd
780,0,46,3.0,1,20.0,0.0,0,0,0,253.0,118.0,82.0,19.70,100.0,70.0,0
2218,1,43,2.0,1,20.0,0.0,0,0,0,226.0,132.5,85.0,26.64,72.0,58.0,0
3251,0,48,3.0,1,20.0,0.0,0,0,0,259.0,129.0,81.0,21.08,75.0,65.0,0
3482,1,38,2.0,0,0.0,0.0,0,1,0,256.0,123.0,92.0,25.42,62.0,82.0,0
2662,0,51,1.0,0,0.0,0.0,0,1,0,285.0,151.5,79.0,26.91,84.0,76.0,0


In [41]:
y_train_over = train_oversampled['tenyearchd']
X_train_over = train_oversampled.drop('tenyearchd', axis = 1)

In [42]:
LR = LogisticRegression(max_iter = 1000)
LR.fit(X_train_over, y_train_over)
pred = LR.predict(X_test)

print('Accuracy: ', LR.score(X_test, y_test))

print('Precision: ', precision_score(y_test,pred))
print('Recall: ', recall_score(y_test,pred))
print('F1: ', f1_score(y_test,pred))

print(classification_report(y_test, pred))

Accuracy:  0.6775956284153005
Precision:  0.2857142857142857
Recall:  0.6229508196721312
F1:  0.3917525773195876
              precision    recall  f1-score   support

           0       0.90      0.69      0.78       610
           1       0.29      0.62      0.39       122

    accuracy                           0.68       732
   macro avg       0.59      0.66      0.59       732
weighted avg       0.80      0.68      0.72       732



In [43]:
confusion_matrix(y_test,pred)

array([[420, 190],
       [ 46,  76]], dtype=int64)

In [44]:
no_CHD = train[train['tenyearchd'] == 0]
yes_CHD = train[train['tenyearchd'] == 1]

no_CHD_undersampled = resample(no_CHD,
                                    replace = False,
                                    n_samples = len(yes_CHD),
                                    random_state = 0)

display(no_CHD_undersampled.shape)
display(yes_CHD.shape)

(435, 16)

(435, 16)

In [45]:
train_undersampled = pd.concat([yes_CHD, no_CHD_undersampled])
train_undersampled.head()

,male,age,education,currentsmoker,cigsperday,bpmeds,prevalentstroke,prevalenthyp,diabetes,totchol,sysbp,diabp,bmi,heartrate,glucose,tenyearchd
3731,1,60,1.0,0,0.0,0.0,0,1,0,220.0,167.5,110.0,30.41,90.0,84.0,1
2528,1,65,2.0,0,0.0,0.0,0,0,1,238.0,122.0,81.0,23.95,67.0,150.0,1
3680,0,47,2.0,1,20.0,0.0,0,1,1,211.0,159.5,82.5,34.08,86.0,250.0,1
4022,1,60,3.0,1,10.0,0.0,0,1,0,250.0,157.0,94.0,29.89,63.0,68.0,1
196,1,48,1.0,1,10.0,0.0,0,0,0,308.0,117.0,76.0,30.85,65.0,54.0,1


In [46]:
y_train_under = train_undersampled['tenyearchd']
X_train_under = train_undersampled.drop('tenyearchd', axis = 1)

In [47]:
LR = LogisticRegression(max_iter = 1000)
LR.fit(X_train_under, y_train_under)
pred = LR.predict(X_test)

print('Precision: ', precision_score(y_test,pred))
print('Recall: ', recall_score(y_test,pred))
print('F1: ', f1_score(y_test,pred))

print(classification_report(y_test, pred))

Precision:  0.2791519434628975
Recall:  0.6475409836065574
F1:  0.39012345679012345
              precision    recall  f1-score   support

           0       0.90      0.67      0.77       610
           1       0.28      0.65      0.39       122

    accuracy                           0.66       732
   macro avg       0.59      0.66      0.58       732
weighted avg       0.80      0.66      0.70       732



In [48]:
confusion_matrix(y_test,pred)

array([[406, 204],
       [ 43,  79]], dtype=int64)